In [4]:
import pickle
import datasets

with open("data/parallel/SARC_PARALLEL", "rb") as file:
    dataset_sarc = pickle.load(file)
with open("data/parallel/ONION_PARALLEL", "rb") as file:
    dataset_onion = pickle.load(file)


anc, res, lab, prob_ = dataset_sarc["slo_anc"], dataset_sarc["slo_res"], dataset_sarc["labels"], dataset_sarc["prob"]
inputs, labels, prob = list(), list(), list()
for a, r_, l_, p in zip(anc, res, lab, prob_):
    for r, l in zip(r_, l_):
        inputs.append(a+r)
        labels.append(l)
        prob.append(p)

text, lab, prob_ = dataset_onion["slo"], dataset_onion["labels"], dataset_onion["prob"]
inputs += text
labels += lab
prob += prob_

anc2, res2 = dataset_sarc["eng_anc"], dataset_sarc["eng_res"]

l = 0
for a in anc2:
    l+=len(a)
print(l)
for r in res2:
    l += sum([len(i) for i in r])
print(l)



inputs2 = list()
for a, r_ in zip(anc2, res2):
    for r in r_:
        inputs2.append(a+"; "+r)
text2 =dataset_onion["eng"]
inputs2 += text2

for t in text2:
    l += len(t)

print(l)

dataset = datasets.Dataset.from_dict({"slo":inputs, "eng":inputs2, "label":labels, "prob":prob})
dataset.sort(column_names="prob")
dataset = dataset.remove_columns(["prob"])
dataset = dataset.select(range(len(inputs)-100000, len(inputs))).shuffle(seed=42)

5284926
12472936
13999888


In [ ]:
dataset["eng"][:10]
from openai import OpenAI
client = OpenAI(api_key="")

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "Translate to Slovene."},
    {"role": "user", "content": dataset["eng"][0]}
  ]
)


In [17]:
import tiktoken
enc = tiktoken.encoding_for_model("gpt-3.5-turbo")
encodings = enc.encode("Translate to Slovene."+dataset["eng"][0])
len(encodings), completion

(42,
 ChatCompletion(id='chatcmpl-9lZ1GOQQNXftpSIaTqAGAVAKD13eS', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Na kakšne načine filmi in TV oddaje ne prikažejo srednješolcev na realističen način? Ne organiziramo velikih zabav in se močno opijamo vsak vikend; To počnemo samo vsak drugi vikend!', role='assistant', function_call=None, tool_calls=None))], created=1721123058, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=72, prompt_tokens=54, total_tokens=126)))

In [2]:
from tqdm import tqdm
from multiprocessing import Pool

def f(i):
    return {"custom_id": f"{i}", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "gpt-4o", "messages": [{"role": "system", "content": "Translate to Slovene."}, {"role": "user", "content": dataset["eng"][i]}],"max_tokens": 1000}}



with Pool(8) as p:
    for _ in tqdm(p.imap_unordered(f, range(16)), total=16):
        pass


  0%|          | 0/16 [00:00<?, ?it/s]

: 

In [ ]:
# BATCH 1
from openai import OpenAI

client = OpenAI(api_key="")
# batch_input_file = client.files.create(file=open("batch1.jsonl", "rb"), purpose="batch")
# batch_input_file_id = batch_input_file.id
# client.batches.create(
#     input_file_id=batch_input_file_id,
#     endpoint="/v1/chat/completions",
#     completion_window="24h",
#     metadata={
#       "description": "translation batch 1"
#     }
# )
# BATCH1 batch_r1gNxBf2oM53aYrb0dZ75UtD file-l5A1sAyvQI1XzmZNhudfIk9l

Batch(id='batch_r1gNxBf2oM53aYrb0dZ75UtD', completion_window='24h', created_at=1721132994, endpoint='/v1/chat/completions', input_file_id='file-MSkB9Zooj5bGNOxiConuRklJ', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1721219394, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'translation batch 1'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [ ]:
client = OpenAI(api_key="")
batch_input_file = client.files.create(file=open("batch2.jsonl", "rb"), purpose="batch")
batch_input_file_id = batch_input_file.id
client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "translation batch 2"
    }
)
# BATCH2 batch_u3rpk4ZnoMTtW8Ndq86OOzwy file-w3EvIpoIOmTIal2QBk04oPH3

Batch(id='batch_u3rpk4ZnoMTtW8Ndq86OOzwy', completion_window='24h', created_at=1721142339, endpoint='/v1/chat/completions', input_file_id='file-nRoZOxOE7ZtcxihZw9yte38T', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1721228739, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'translation batch 2'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [ ]:
client = OpenAI(api_key="")
batch_input_file = client.files.create(file=open("batch3.jsonl", "rb"), purpose="batch")
batch_input_file_id = batch_input_file.id
client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "translation batch 3"
    }
)
# BATCH3 batch_szhrmjRxScxss0H2FtyZkbNa file-SrGv4yqXAXwcqOjztF6TjE2j

Batch(id='batch_szhrmjRxScxss0H2FtyZkbNa', completion_window='24h', created_at=1721142353, endpoint='/v1/chat/completions', input_file_id='file-vWKFVwfDj7kQXCRhs1wHNb7L', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1721228753, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'translation batch 3'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [ ]:
client = OpenAI(api_key="")
batch_input_file = client.files.create(file=open("batch4.jsonl", "rb"), purpose="batch")
batch_input_file_id = batch_input_file.id
client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "translation batch 4"
    }
)
# BATCH4 batch_njaPSZxKSN7OlIHDreMxxwJz file-BnwE9xDlllDeNmDy1UxkJTWf

Batch(id='batch_njaPSZxKSN7OlIHDreMxxwJz', completion_window='24h', created_at=1721142382, endpoint='/v1/chat/completions', input_file_id='file-fumJZQNzxIBelmyP9dcEqvnO', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1721228782, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'translation batch 4'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [ ]:
from openai import OpenAI

client = OpenAI(api_key="")

print(client.batches.list(limit=10))


SyncCursorPage[Batch](data=[Batch(id='batch_njaPSZxKSN7OlIHDreMxxwJz', completion_window='24h', created_at=1721142382, endpoint='/v1/chat/completions', input_file_id='file-fumJZQNzxIBelmyP9dcEqvnO', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1721147359, error_file_id=None, errors=None, expired_at=None, expires_at=1721228782, failed_at=None, finalizing_at=1721145602, in_progress_at=1721142399, metadata={'description': 'translation batch 4'}, output_file_id='file-BnwE9xDlllDeNmDy1UxkJTWf', request_counts=BatchRequestCounts(completed=25000, failed=0, total=25000)), Batch(id='batch_szhrmjRxScxss0H2FtyZkbNa', completion_window='24h', created_at=1721142353, endpoint='/v1/chat/completions', input_file_id='file-vWKFVwfDj7kQXCRhs1wHNb7L', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1721147303, error_file_id=None, errors=None, expired_at=None, expires_at=1721228753, failed_at=None, finalizing_at=1721145603, 

In [4]:
#dir(file_response)
#file_response.write_to_file("batch1_out.jsonl")
file_response = client.files.content("file-w3EvIpoIOmTIal2QBk04oPH3")
file_response.write_to_file("batch2_out.jsonl")

file_response = client.files.content("file-SrGv4yqXAXwcqOjztF6TjE2j")
file_response.write_to_file("batch3_out.jsonl")

file_response = client.files.content("file-BnwE9xDlllDeNmDy1UxkJTWf")
file_response.write_to_file("batch4_out.jsonl")


In [5]:
import json


translations = [None for _ in range(100000)]
for x in range(1,5):
    with open(f'batch{x}_out.jsonl', 'r') as json_file:
        json_list = list(json_file)

    for json_str in json_list:
        result = json.loads(json_str)
        index, text = int(result['custom_id']), result['response']['body']['choices'][0]['message']['content']
        translations[index] = text


translations
import pickle
with open("data/gpt/translations_", "wb") as file:
    pickle.dump(translations, file)




In [9]:
dataset = dataset.remove_columns(["slo"])
dataset = dataset.add_column("slo", translations)  # type: ignore

with open("data/parallel/combined-final", "wb") as file:
    pickle.dump(dataset[:], file)


100000